**Clone repository**

In [ ]:
!git clone https://github.com/Uncategorised/first-order-model

Cloning into 'first-order-model'...
remote: Enumerating objects: 246, done.
remote: Total 246 (delta 0), reused 0 (delta 0), pack-reused 246
Receiving objects: 100% (246/246), 71.46 MiB | 22.54 MiB/s, done.
Resolving deltas: 100% (121/121), done.


In [ ]:
cd first-order-model

/content/first-order-model


**Mount your Google drive folder on Colab**

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


**Load driving video and source image**

In [ ]:
import imageio
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.animation as animation
from skimage.transform import resize
from IPython.display import HTML
import warnings
import os
warnings.filterwarnings("ignore")

image_target = '/content/gdrive/My Drive/first-order-motion-model/monalisa_vinci.png'
video_source = '/content/gdrive/My Drive/first-order-motion-model/source_trump.mp4'

output_file_name = os.path.splitext(image_target)[0]

source_image = imageio.imread(image_target)
reader = imageio.get_reader(video_source)

#Resize image and video to 256x256

source_image = resize(source_image, (256, 256))[..., :3]

fps = reader.get_meta_data()['fps']
driving_video = []
try:
    for im in reader:
        driving_video.append(im)
except RuntimeError:
    pass
reader.close()

driving_video = [resize(frame, (256, 256))[..., :3] for frame in driving_video]

def display(source, driving, generated=None):
    fig = plt.figure(figsize=(8 + 4 * (generated is not None), 6))

    ims = []
    for i in range(len(driving)):
        cols = [source]
        cols.append(driving[i])
        if generated is not None:
            cols.append(generated[i])
        im = plt.imshow(np.concatenate(cols, axis=1), animated=True)
        plt.axis('off')
        ims.append([im])

    ani = animation.ArtistAnimation(fig, ims, interval=50, repeat_delay=1000)
    plt.close()
    return ani
    

HTML(display(source_image, driving_video).to_html5_video())

**Create a model and load checkpoints**

In [ ]:
from demo import load_checkpoints
generator, kp_detector = load_checkpoints(config_path='config/vox-256.yaml', 
                            checkpoint_path='/content/gdrive/My Drive/first-order-motion-model/vox-cpk.pth.tar')

**Perform image animation**

In [ ]:
from demo import make_animation
from skimage import img_as_ubyte

predictions = make_animation(source_image, 
                             driving_video, 
                             generator, 
                             kp_detector, 
                             relative=True)

#save resulting video
imageio.mimsave('../generated_{}.mp4'.format(output_file_name), 
                [img_as_ubyte(frame) for frame in predictions],
                fps=fps)
#video can be downloaded from /content folder

HTML(display(source_image, driving_video, predictions).to_html5_video())

100%|██████████| 211/211 [00:26<00:00,  7.92it/s]
